# Analysis of a Dice Tower

In which I investigate the randomness properties of dice rolled using a home-made dice tower, with the techniques given by Donald Knuth in *The Art of Computer Programming, Vol. 2: Seminumerical Algorithms.*

In [1]:
import math
import scipy.stats
import matplotlib.pyplot as plt

# Number formatting fuctions
def f(number): return f'{round(number, 3)}'
def p(number): return f'{round(number * 100, 1)}%'

# Factorial!
def fact(n):
    'n!'
    if n <= 1: return 1
    else:      return n * fact(n - 1)

In [2]:
def count(sample, n):
    'Return counts of each element of sample, '
    'where each sample is 0..n'
    count = [0] * n
    for elt in sample:
        count[elt - 1] += 1
    return count

The Stirling numbers ${n \brace m}$ can be computed with the function

$$
S(n,m) =
\begin{cases}
               1 & \textrm{if } m = 1 \\
               1 & \textrm{if } n = m \\
               0 & \textrm{if } n < m \\
               m S(n-1,m) + S(n-1,m-1) & \textrm{otherwise}
\end{cases}
$$

See https://people.sc.fsu.edu/~jburkardt/py_src/polpak/stirling2.py.

In [3]:
def Stirling(n, m):
    if   m == 1: return 1
    elif m == n: return 1
    elif n < m:  return 0
    else:
        return (
            (m * Stirling(n - 1, m))
            + Stirling(n - 1, m - 1)
        )

In [4]:
def by_n(sequence, n):
    'iterate over n-element slices of sequence'
    for i in range(0, len(sequence), n):
        yield list(sequence[i:i+n])

def by_n_overlap(sequence, n):
    'iterate over every n-element slice of sequence'
    for i in range(0, len(sequence) + 1 - n):
        yield list(sequence[i:i+n])


I got into a discussion about dice towers on a forum a while back, when someone posted plans for building one. I said, "Hey, I could do that!" Then, I tossed out the plans, got some blue painter's tape and a former priority mail box and slapped together my very own dice tower. (Pro Tip: make sure the lowest baffle is slanted towards the front so the dice are ejected into the tray rather than left in the bottom of the tower.)

Then, as one does, I asked myself, "How random is this thing?"

Ignoring the philosophical side of that question, I knew there were sets of test suites for computer pseudo-random number generators. Things with names like DIEHARD. And the one from the National Institute of Standards and Technology. They are a little important if you want to generate cryptographically secure random numbers, for instance. But I didn't want to have to fight with converting something like that to what I wanted. Basically, I'm lazy.

But the grand-daddy of those suites lives in Donald Knuth's epic *The Art of Computer Programming, Volume II: Seminumerical Algorithms.* That seemed like a good place to go.

So I rolled some dice, wrote some Python, rolled some more dice, and came up with ...

## Methodology

The dice tower in question was assembled from a USPS priority mail box (used) and blue painter's tape in about 10 minutes, without the use of any form of plans or instructions. It has three baffles with an additional ramp to convice the dice to exit the tower, since the baffles are in the wrong positions.

The dice are three pink d6 of unknown origin, about 10-12mm, maybe. They had the most rounded edges of my dice collection and were not very large

### Dice rolling

The dice were rolled one at a time by holding the 1 pip up approximately 25mm above the dice tower top opening and releasing them under gravity. Three dice were rolled in each trial---the value chosen is the value initially rolled on the dice, which may be changed by the impact of a later dice. The sample sequence consists of 999 rolls.

In addition to the sample being tested, I created several other sequences for comparison.

`rolls` is the result of rolling dice.

`sorted_rolls` is `rolls`, sorted, as a control test. (Obviously, it would be locally very non-random, but it satisfies basic tests of a uniform random sequence.)

`duplicated` is `rolls`, with each element duplicated, like [3, 3, 1, 1, ...], as another control test. This sequence is also non-random at a small scale, but should pass for a uniform random sequence to simple tests.

`repetition` is a sequence of repetitions of 1-6, with a length equal to the length of rolls, as one control test.

There are many kinds of randomness, called "distributions", that describe the probabilities of occurrence of the different possible outcomes. The value of a single dice roll is, or should be, from a "uniform discrete distribution".

"Discrete" means that the values are, for example, the integers 1 to 6, as opposed to a "continuous" distribution from the real numbers between 0 and 1.

"Uniform" means that the probability of all of the values is the same; a 1 is rolled 1 out of 6 times, as is 2, 3, or 4. Or 5 or 6.

`normal` is a sequence of discrete values from a truncated normal distribution over [1,6], as a random, but non-uniformly distributed control test.

`uniform` is a sequence of values from scipy.stats' uniform distribution, which are $0 \leq x \lt 1$ and then moved into the range [1,6] by $\lfloor{6x}\rfloor + 1$. This is another control test which should be positive.

In [5]:
rolls = list(map(int,
    '313141462543662425625132222521641644463233423221345122666465'
    '366423246553145466524122645362513543543326655236561436422616'
    '611613112624535652352564164532146152441526435512423115163413'
    '332151643426566431425635221141124661516423215513532534554326'
    '522343666132235566231264113132446115521541113216412524426615'
    '342553344426541534655246136631251362246243634634433312125265'
    '224612651221552336624611164243633621245115452152444416362232'
    '156456653234361315626124622261421354415451442424356412121533'
    '225316344434454125162115662514312255654355331332432154143522'
    '634454125523651352225262335662232312136343135163465454621524'
    '564152663235435612156423544333544613412344463522624634261616'
    '615566414143361664411553343521515552363331315216235254625326'
    '434346612531512565511563446423646416365322431663354221225145'
    '543654126651535163436662341165266516511552615233141216621142'
    '513442634663325356235441352552432664231266451126344635165441'
    '426555345332551566551464125335125465266111433513342644251636'
    '343153241542132252165135336416443466334'
))

repetition = [1,2,3,4,5,6] * (len(rolls) // 6)

sorted_rolls = list(rolls)
sorted_rolls.sort()

duplicated = [
    item
    for sublist in map(lambda x: [x, x], rolls)
    for item in sublist
]

normal = scipy.stats.truncnorm(a=1/3, b=6/3, scale=3).rvs(len(rolls)).round().astype(int)

uniform = list(map(lambda x: math.floor(6*x) + 1, scipy.stats.uniform.rvs(size=len(rolls))))

len(uniform)

999

## Tests

The most basic test possible tries to find out if the sequence matches a uniform random distribution.

### Chi-square

The $\chi^2$ test is a good ol' staple of statistical statistics. It compares an experimental sequence with an expected random distribution.

If you want to know more about the $\chi^2$ test, there are about a billion videos on YouTube explaining it. Also, it can be found in every statistic textbook in the world.

For my purposes, there are a couple things that are important about the $\chi^2$ test:

First, it works on discrete distributions, like dice rolls. The first step is to count the number of ones rolled, the number of twos rolled, and so on. These counts are what is compared with the expected distribution, producing a value, the `$\chi^2$ statistic`.

This statistic can then be run through the $\chi^2$ distribution function, with an appropriate number of degrees of freedom, to produce a percentage value, called the "p value".

The p value represents the probability of getting a result as "extreme" as the sample from the expected random distribution.

The second thing about the $\chi^2$ test is that I'm using it here as a "two-sided" test. Normally, as in all of those other videos, the test is used to determine if your experimental sequence is too extreme in one direction. For testing randomness, Knuth and I are using it to test if the sequences is too extreme in both directions. I'm sure there are some mathematical complications I'm missing, but I intend to simply follow Knuth's rule of thumb:

If the p value is below 1% or above 99%, there is something wrong with the sequence. If it is below 5% or above 95%, there is something suspicious about the sequence.

In [6]:
def chi2test(sample, pvalue=True, f_exp=None):
    'Perform the chi-squared test on counts, '
    'normally returning the pvalue but optionally '
    'the statistic value.'
    result = scipy.stats.chisquare(
        sample,
        f_exp = f_exp
    )
    if pvalue:
        return result.pvalue
    else:
        return result.statistic

Applying the $\chi^2$ test to the test sequences above gives:

| Sequence | $\chi^2$ |
|:--------:|:-----------------:|
| `rolls` | {{ p(chi2test(count(rolls, 6))) }} |
| `repetition` | {{ p(chi2test(count(repetition, 6))) }} |
| `sorted_rolls` | {{ p(chi2test(count(sorted_rolls, 6))) }} |
| `duplicated` | {{ p(chi2test(count(duplicated, 6))) }} |
| `normal` | {{ p(chi2test(count(normal, 6))) }} |
| `uniform` | {{ p(chi2test(count(uniform, 6))) }} |

**The $\chi^2$ *pvalue* for the `rolls` is {{ p(chi2test(count(rolls, 6))) }}**, which is bad. In fact, it seems to be *too* uniform. I'm not sure what's going on there.

**The $\chi^2$ value for `sorted_rolls` is also {{ p(chi2test(count(sorted_rolls, 6))) }},** and **the $\chi^2$ value for `duplication` is {{ p(chi2test(count(duplicated, 6))) }},** both fairly close to the value of `rolls` because they are derived from `rolls` in a way that preserves its distribution.

**The $\chi^2$ value for the `repetition` is {{ p(chi2test(count(repetition, 6))) }} and that for `normal` is {{ p(chi2test(count(normal, 6))) }}.** Those are obviously bad.

Finally, **the $\chi^2$ value for `uniform` is {{ p(chi2test(count(uniform, 6))) }},** which is fine. As it should be, since it's taken from a decent pseudo-random number generator.

Knuth suggests making tests on smaller sub-sequences of the overall random sequence being tested, in order to look at local behavior.

In [7]:
def sub_chi2test(samples, n, sides=6, pvalue=True):
    'Compute χ^2 values for n-element subsequences of lst '
    'assuming elements are 1..sides'
    return [
        chi2test(count(sublst, sides), pvalue=pvalue)
        for sublst in by_n(samples, n)
    ]

**The $\chi^2$ *pvalue* of each 60-element subsequences of `rolls` is {{ ', '.join(map(p, sub_chi2test(rolls, 60))) }};** most of the values are acceptable although some are a little sketchy. **The same values for `repetition` is {{ ', '.join(map(p, sub_chi2test(repetition, 60))) }},** which is as bad as the value of the whole sequence. On the other hand, if we break it at 61, the subsequences are {{ ', '.join(map(p, sub_chi2test(repetition, 61))) }}, which is bad in a different way.

**The $\chi^2$ *pvalue* of `sorted_rolls` is {{ p(chi2test(count(sorted_rolls, 6))) }}, the same as the unsorted list, but the subsequence test is {{ ', '.join(map(p, sub_chi2test(sorted_rolls, 60))) }}, clearly indicating something is wrong.**

**The subsequence test for `normal` is {{ ', '.join(map(p, sub_chi2test(normal, 60))) }},** which seems suspicious.

### Kolmogorov-Smirnov

Now, you could look at the $\chi^2$ p values for each of the subsequences in the expimental sample, but it is rather useless. Knuth's recomendation for producing a meaningful value uses the Kolmogorov-Smirnov test.

(Yes, there should be vodka for this.)

In [8]:
def kstest(samples, cdf=scipy.stats.uniform.cdf):
    'Compute KS test for samples, compared to distribution cdf'
    return scipy.stats.kstest(samples, cdf).pvalue

The Kolmogorov-Smirnov test is fundamentally similar to the $\chi^2$ test. It compares an experimental sequence to a distribution and produces a p value.

However, there's a problem. The problem, as Knuth notes, is

> The Kolmogorov-Smirnov test (KS test) may be used when $F(x)$ [the experimental sequence] has no jumps.

KS is useful for continuous distributions, such as what are typically produced by computer pseudo-random number generators---typically floating point numbers between 0 and 1---while the $\chi^2$ test is useful for discrete distributions such as this case where the result is an integer between 1 and 6  meaning that they are entirely jumps. The bottom line is that, in the case of dice, the $\chi^2$ test is more appropriate. 

Knuth suggests one good use of Kolmogorov-Smirnov with dice rolling: if I break up the rolls into 60-element subsequences, perform the $\chi^2$ test on the subsequences, I can then use the KS test to combine the results. This *is* a reasonable thing to do, because the statistic result of the $\chi^2$ test comes from the $\chi^2$ distribution, which is continuous.

In order to combine the $\chi^2$ results in this way, we need to get the $\chi^2$ statistics and compare their distributions to the $\chi^2$ cumulative distribution function with the appropriate degrees of freedom (5 in the case of 6-sided dice rolls). (And no, you are not expected to understand that last remark. I certainly don't.)

The result is another p value.

In [9]:
def ks_combine_chi2(samples, n, sides=6):
    'Compute χ^2 values for n-element subsequences of lst, '
    'then combine them with the KS test using χ^2 distribution'
    df = max(samples) - 1
    return kstest(
        sub_chi2test(samples, n, sides=sides, pvalue=False),
        cdf=lambda x: scipy.stats.chi2.cdf(x, df)
    )

| Sequence | Combined $\chi^2$ |
|:--------:|:-----------------:|
| `rolls` | {{ p(ks_combine_chi2(rolls, 30)) }} |
| `repetition` | {{ p(ks_combine_chi2(repetition, 30)) }} |
| `sorted_rolls` | {{ p(ks_combine_chi2(sorted_rolls, 30)) }} |
| `duplicated` | {{ p(ks_combine_chi2(duplicated, 30)) }} |
| `normal` | {{ p(ks_combine_chi2(normal, 30)) }} |
| `uniform` | {{ p(ks_combine_chi2(uniform, 30)) }} |

**Combining the results of the 30-element $\chi^2$ subsequence test on `rolls` produces {{ p(ks_combine_chi2(rolls, 30)) }} and the `uniform` sample produces {{ p(ks_combine_chi2(uniform, 30)) }}, while the `repetition` sample produces {{ p(ks_combine_chi2(repetition, 30)) }}, the `sorted_rolls` sample produces {{ p(ks_combine_chi2(sorted_rolls, 30)) }}, the `duplicated` sample is {{ p(ks_combine_chi2(duplicated, 30)) }}, and the `normal` sample produces {{ p(ks_combine_chi2(normal, 30)) }}.** The first two are good, the rest are not.

### Empirical tests

Knuth presents 10 "empirical" tests "applied to sequences in order to investigate their randomness".

For an example, the first of these tests is the *Equidistribution test* or *Frequency test*, that the numbers from the sequence are uniformly distributed across their range. One alternative is to plot a histogram.

**Histogram for `rolls`**

{{ _ = plt.hist(x=rolls, bins=6, color='#0504aa', rwidth=0.85) }}

**Histogram for `repetition`**

{{ _ = plt.hist(x=repetition, bins=6, color='#0504aa', rwidth=0.85) }}

At least it's lovely and even.

**Histogram for `sorted_rolls`**

{{ _ = plt.hist(x=sorted_rolls, bins=6, color='#0504aa', rwidth=0.85) }}

Weirdly similar to `rolls`.

**Histogram for `duplicated`**

{{ _ = plt.hist(x=duplicated, bins=6, color='#0504aa', rwidth=0.85) }}

A slightly more emphatic version of `rolls`.

**Histogram for `normal`**

{{ _ = plt.hist(x=normal, bins=6, color='#0504aa', rwidth=0.85) }}

Very definitely odd.

**Histogram for `uniform`**

{{ _ = plt.hist(x=uniform, bins=6, color='#0504aa', rwidth=0.85) }}

But Knuth points out that determining whether the sequence elements are uniformly distributed across their range is one of the purposes of the $\chi^2$ and KS tests. So that seems rather taken care of.

#### Serial test

This test determines whether successive rolls from the sequence are uniformly distributed in an independent matter. It does this by counting how often given sub-sequences (i.e. $s_i, ..., s_{i+n}$ for a sub-sequence of length $n+1$) and then running the $\chi^2$ test against the counts.

In [10]:
def inc_bin(ary, n, loc):
    elt = 0
    for i in loc:
        elt = (elt * n) + (i - 1)
    ary[elt] += 1

def serial_test(sample, n, sides=6):
    ary = [0] * (sides ** n)
    for loc in by_n_overlap(sample, n):
        inc_bin(ary, sides, loc)
    # print(ary)
    return chi2test(ary)

This test is ideal for picking out sequences like `duplicated`, which would have many more doubles like 2,2 and 4,4 than it ought to.

I chose to look at sequences of length two and sequences of length three.

The serial test values for the various sequences are:

| Sequence | Length 2 | Length 3 |
|:--------:|:--------:|:--------:|
| `rolls`  | {{ p(serial_test(rolls, 2)) }} | {{ p(serial_test(rolls, 3)) }} |
| `repetition`  | {{ p(serial_test(repetition, 2)) }} | {{ p(serial_test(repetition, 3)) }} |
| `sorted_rolls`  | {{ p(serial_test(sorted_rolls, 2)) }} | {{ p(serial_test(sorted_rolls, 3)) }} |
| `duplicated`  | {{ p(serial_test(duplicated, 2)) }} | {{ p(serial_test(duplicated, 3)) }} |
| `normal` | {{ p(serial_test(normal, 2)) }} | {{ p(serial_test(normal, 3)) }} |
| `uniform` | {{ p(serial_test(uniform, 2)) }} | {{ p(serial_test(uniform, 3)) }} |

**The serial test does a very good job of separating `rolls` and `uniform` from samples with non-uniformly random local behavior.** Unfortunately, serial lengths much above 3 lead to large numbers of bins for the $\chi^2$ and reduced counts in the bins.

#### Gap test

The gap test examines the length of "gaps" between two occurrences of a value.

I'm totally ignoring Knuth's algorithm for this because it is intended for continuous values and because I just don't see what he's getting at for them.

The probability distribution that the gap lengths should be compared against is not a simple uniform distribution, but rather given by a formula:

$$
p_r = 
\begin{cases}
p (1 - p)^r & \textrm{if } 0 \leq r \lt t \\
(1 - p)^r     & \textrm{if } r = t
\end{cases}
$$

for $0 \leq r \leq t$ and where $p$ is the probability of a dice rolling a given side, $1/\textrm{bins}$. When $0 \leq r \lt t$, $r$ the index into the counts and $t$ is a catch-all gap length to limit the number of counts---any gap longer than $t$ is counted in $t$.

In [11]:
def count_gaps(sample, bins):
    'Count the gaps between subsequent occurances '
    'of each element of sample. The elements of the '
    'sample should be in 0..bins.'
    counts = {}
    seen = {}
    for i in range(0, bins): seen[i] = 0
    for elt in sample:
        for i in range(0, bins):
            if i == (elt - 1):
                # update counts for current elt
                if seen[i] in counts:
                    counts[seen[i]] += 1
                else:
                    counts[seen[i]] = 1
                seen[i] = 0
            else:
                # increment seen for non-current elt
                seen[i] += 1
    # finish counts at end of sample
    for i in seen:
        if seen[i] in counts:
            counts[seen[i]] += 1
        else:
            counts[seen[i]] = 1
    return [
        counts[k] if k in counts else 0
        for k in range(0, max(counts) + 1)
    ]

def gap_test(sample, bins):
    'Perform the gap test on sample, returning a '
    'pvalue. The elements of the sample should be '
    'in 0..bins.'
    counts = count_gaps(sample, bins)
    p = 1 / bins
    n = len(counts)
    s = sum(counts)
    f_exp = [
        s * p * pow(1 - p, i)
        for i in range(0, n)
    ]
    f_exp[n - 1] = s * pow(1 - p, n - 1)
    return chi2test(
        counts,
        f_exp = f_exp
    )

The gap test values for the various samples are:

| Sequence | Gap test |
|:--------:|:--------:|
| `rolls`  | {{ p(gap_test(rolls, 6)) }} |
| `repetition`  | {{ p(gap_test(repetition, 6)) }} |
| `sorted_rolls`  | {{ p(gap_test(sorted_rolls, 6)) }} |
| `duplicated`  | {{ p(gap_test(duplicated, 6)) }} |
| `normal` | {{ p(gap_test(normal, 6)) }} |
| `uniform` | {{ p(gap_test(uniform, 6)) }} |

Once again, **the gap test does a good job of differentiating `rolls` from the other samples,** whose gaps do not match the expected distribution for various reasons.

#### Poker test

The poker test is kind of an extension of the serial test. The basic idea is to take each five successive elements of the sample and count the numbers of five-of-a-kind, four-of-a-kind, full house, etc., and make sure the counts are from the right distribution.

A slightly easier version is simply to count the number of distinct values in the hand of five rolls: 1 is five-of-a-kind,  3 is either two pairs and a single or three of a kind and two singles, etc.

The probability distribution that the poker test should be compared against is:

$$
p_r = \frac{\Pi_{0 \leq i \leq r + 1}(d - i)}{d^k}{k \brace r}
$$

Where $d$ is the number of sides on the dice, $0 \leq r \lt k$ and $k$ is the size of the poker hand.

The weird looking thing on the right is the notation for the Stirling Numbers of the second kind, which gives how many ways there are of dividing up a set of $k$ objects into $r$ nonempty subsets.

In [12]:
def poker_dist(d, k, r):
    prod = 1
    for t in range(d, d - r, -1):
        prod *= t
    return Stirling(k, r) * prod / pow(d, k)

def poker_test(sample, k, sides=6):
    sequence = by_n_overlap(sample, k)
    counts = count(
        map(lambda elt: len(set(elt)), sequence),
        k
    )
    return chi2test(
        counts,
        f_exp = [
            sum(counts) * poker_dist(sides, k, r)
            for r in range(1, sides)
        ]
    )

| Sequence | Poker test |
|:--------:|:--------:|
| `rolls`  | {{ p(poker_test(rolls, 5)) }} |
| `repetition`  | {{ p(poker_test(repetition, 5)) }} |
| `sorted_rolls`  | {{ p(poker_test(sorted_rolls, 5)) }} |
| `duplicated`  | {{ p(poker_test(duplicated, 5)) }} |
| `normal` | {{ p(poker_test(normal, 5)) }} |
| `uniform` | {{ p(poker_test(uniform, 5)) }} |


#### Coupon collector's test

Picture someone trying to get a complete set of 1 to 6; how many rolls would they have to look at? This test looks at the length of the sequences in the sample needed to see all 6 values and compares it to the distribution:

$$
p_r =
\begin{cases}
\frac{d!}{d^r}{r - 1 \brace d - 1} & \textrm{if } d \leq r \lt t \\
1 - \frac{d!}{d^{t-1}}{t - 1 \brace d} & \textrm{if } r = t
\end{cases}
$$

where $d$ is the number of sides (coincidentally meaning that the length of the sequences must be at least $d$), $r$ is $d \leq r \leq t$, and $t$ is the maximum length of sequences so that any sequences longer are counted in $r = t$.

In [13]:
def coupon_count(sample, sides):
    count  = {}
    occurs = [False] * sides
    length = 0
    for x in sample:
        occurs[x - 1] = True
        length += 1
        if all(occurs):
            if length in count:
                count[length] += 1
            else:
                count[length] = 1
            length = 0
            occurs = [False] * sides
    # build result array
    # min len(result) = 2 * sides
    # skip the first sides elements
    # because they're always 0
    counts = max(count) + 1 - sides
    min_counts = 2 * sides
    result = [
        count[k] if k in count else 0
        for k in range(
            sides,
            sides + max(counts, min_counts)
        )
    ]
    # maximum len(result) = 5 * sides
    max_counts = 5 * sides
    if len(result) > max_counts:
        s = sum(result[max_counts:])
        result = result[:max_counts]
        result[max_counts - 1] += s
    return result
    
def coupon_test(sample, sides):
    count  = coupon_count(sample, sides)
    total  = sum(count)
    counts = len(count)
    f_exp = [
        total * ((fact(sides) / pow(sides, r)) * Stirling(r - 1, sides - 1))
        for r in range(sides, counts + sides - 1)
    ]
    f_exp.append(
        total * (1 - ((fact(sides) / pow(sides, counts - 1)) * Stirling(counts - 1, sides)))
    )
    return chi2test(count, f_exp=f_exp)

| Sequence | Coupon collector's test |
|:--------:|:--------:|
| `rolls`  | {{ p(coupon_test(rolls, 6)) }} |
| `repetition`  | {{ p(coupon_test(repetition, 6)) }} |
| `sorted_rolls`  | {{ p(coupon_test(sorted_rolls, 6)) }} |
| `duplicated`  | {{ p(coupon_test(duplicated, 6)) }} |
| `normal` | {{ p(coupon_test(normal, 6)) }} |
| `uniform` | {{ p(coupon_test(uniform, 6)) }} |


#### Permutation test

In [14]:
def max_loc(perm, end):
    loc = 0
    max = perm[0]
    for i in range(1, end):
        if perm[i] > max:
            loc = i
            max = perm[i]
    return loc

def perm_int(perm):
    f = 0
    for r in range(len(perm), 1, -1):
        s = max_loc(perm, r) + 1
        f = r * f + s - 1
        perm[r - 1], perm[s - 1] = perm[s - 1], perm[r - 1]
    return f

def permutation_count(sample, n):
    count = [0] * fact(n)
    for seq in by_n(sample, n):
        pi = perm_int(seq)
        count[pi] += 1
    return count

def permutation_test(sample, n):
    return chi2test(permutation_count(sample,n))

The permutation test is not applicable (for some reason) to discrete samples. This is the result on `rolls`, etc:

| Sequence | Permutation test |
|:--------:|:--------:|
| `rolls`  | {{ p(permutation_test(rolls, 3)) }} |
| `repetition`  | {{ p(permutation_test(repetition, 3)) }} |
| `sorted_rolls`  | {{ p(permutation_test(sorted_rolls, 3)) }} |
| `duplicated`  | {{ p(permutation_test(duplicated, 3)) }} |
| `normal` | {{ p(permutation_test(normal, 3)) }} |
| `uniform` | {{ p(permutation_test(uniform, 3)) }} |

However, if you compute the permutation test on a uniform random distribution of numbers between 0 and 1, you get
{{ p(permutation_test(list(scipy.stats.uniform.rvs(size=len(rolls))), 3)) }}.

#### Run test

In [15]:
def runs_up(sample, n):
    count = [0] * n
    k = 1
    i = 0
    while i < len(sample) - 1:
        if sample[i] <= sample[i+1]:
            k += 1
            i += 1
        else:
            if k < n:
                count[k - 1] += 1
            else:
                count[n - 1] += 1
            k = 1
            i += 2
    if k < n:
        count[k - 1] += 1
    else:
        count[n - 1] += 1
    return count

def runs_fexp(s, n):
    return [
        s * ((1 / fact(r)) - (1 / fact(r + 1)))
        for r in range(1, n + 1)
    ]

def runs_test(sample, n):
    c = runs_up(sample, n)
    s = sum(c)
    f = runs_fexp(s, n)
    return chi2test(c, f_exp=f)

Likewise, the runs test is not applicable to discrete samples.

| Sequence | Permutation test |
|:--------:|:--------:|
| `rolls`  | {{ p(runs_test(rolls, 5)) }} |
| `repetition`  | {{ p(runs_test(repetition, 5)) }} |
| `sorted_rolls`  | {{ p(runs_test(sorted_rolls, 5)) }} |
| `duplicated`  | {{ p(runs_test(duplicated, 5)) }} |
| `normal` | {{ p(runs_test(normal, 5)) }} |
| `uniform` | {{ p(runs_test(uniform, 5)) }} |

The result of the runs test applied to a sample from a uniform random distribution between 0 and 1 is {{ p(permutation_test(list(scipy.stats.uniform.rvs(size=len(rolls))), 3)) }}.

#### Serial correlation test

In [16]:
def corr_rng(n):
    mu = -1 / (n - 1)
    sig = (1 / (n - 1)) * math.sqrt((n * (n - 3)) / (n + 1))
    return (mu - (2 * sig), mu + (2 * sig))

def pc(sample):
    s = corr_rng(len(sample))
    return f'[{f(s[0])},{f(s[1])}]'

def correlation_coef(sample, n):
    corr = scipy.stats.pearsonr(
        sample,
        [sample[(i+n) % len(sample)] for i in range(0, len(sample))]
    )[0]
    rng = corr_rng(len(sample))
    return (corr, rng, rng[0] < corr and corr < rng[1])

def cor(sample, n):
    cr = correlation_coef(sample, n)
    return f'{cr[2] and "Valid" or "Invalid"} ({f(cr[0])})'

def failures(sample):
    coefs = [
        correlation_coef(sample, i)
        for i in range(1, len(sample))
    ]
    tot = len(coefs)
    fails = list(filter(lambda x: not x[2], coefs))
    bad = len(fails)
    worstd = min(fails, key=lambda elt: elt[0], default=(0.0, 0, 0))
    worstu = max(fails, key=lambda elt: elt[0], default=(0.0, 0, 0))
    return f'{p(bad / tot)} ({f(worstd[0])}/{f(worstu[0])})'


| Sequence | 1 | 2 | 4 | 6 | $2\sigma$ | Failures |
|:--------:|:-:|:-:|:-:|:-:|:-:|:-:|
| `rolls`  | {{ cor(rolls, 1) }} | {{ cor(rolls, 2) }} | {{ cor(rolls, 4) }} | {{ cor(rolls, 6) }} | {{ pc(rolls) }} | {{ failures(rolls) }} |
| `repetition`  | {{ cor(repetition, 1) }} | {{ cor(repetition, 2) }} | {{ cor(repetition, 4) }} | {{ cor(repetition, 6) }} | {{ pc(repetition) }} | {{ failures(repetition) }} |
| `sorted_rolls`  | {{ cor(sorted_rolls, 1) }} | {{ cor(sorted_rolls, 2) }} | {{ cor(sorted_rolls, 4) }} | {{ cor(sorted_rolls, 6) }} | {{ pc(sorted_rolls) }} | {{ failures(sorted_rolls) }} |
| `duplicated`  | {{ cor(duplicated, 1) }} | {{ cor(duplicated, 2) }} | {{ cor(duplicated, 4) }} | {{ cor(duplicated, 6) }} | {{ pc(duplicated) }} | {{ failures(duplicated) }} |
| `normal`  | {{ cor(normal, 1) }} | {{ cor(normal, 2) }} | {{ cor(normal, 4) }} | {{ cor(normal, 6) }} | {{ pc(normal) }} | {{ failures(normal) }} |
| `uniform`  | {{ cor(uniform, 1) }} | {{ cor(uniform, 2) }} | {{ cor(uniform, 4) }} | {{ cor(uniform, 6) }} | {{ pc(uniform) }} | {{ failures(uniform) }} |


In [17]:
GameScience_D6 = list(map(int,
                          '5654461541632141222466411462244526533' #1
                          '1536414413156224364556121314442542214' #3
                          '5565263636153525412223666262125143526' #5
                          '4463345361613523636155553646365263163' #7
                          '4411531312233362356564514224564325365' #9
                          '5441214245246441662112653225453266422' #11
                          '4315246352611236625614633613436211246' #13
                          '4225562515626162661414364265116461661' #15
                          '6552313133432614114565162212452225341' #17
                          '1244135456542514346242461662625145166' #19
                          '2133534136636446663331356335342662135' #21
                          '1244235126254152336233616124324321545' #23
                          '2122361256166165455522611615645631656' #25
                          '4351351432664462241126252616556561232' #27
                          '5465241351224163342532435345233416363' #29
                          '246416415215562361554542566664415433'  #31
                          '333551254461453545661534651364626262'  #33
                          '4263225322214134514416325624411643154' #2
                          '5324363521162541615321465463525331414' #4
                          '2142441153541456243564361332155556212' #6
                          '5511266152114451356411155465454435435' #8
                          '6424531235615663565454562562145363154' #10
                          '5544214345136224266461525522624453261' #12
                          '6441312454236223525361141142544566641' #14
                          '6551644132144544343652112121364326461' #16
                          '1461435515421661623336143143356454421' #18
                          '6262124624142454442155153255136413234' #20
                          '5354131412242533113444524532141452426' #22
                          '6555622433145514352222344423663566154' #24
                          '6123464414426255536551516222243534644' #26
                          '6455211413665212456611642154641135616' #28
                          '161532146351453322522626431142143461'  #30
                          '421312126135415336225213356515341166'  #32
                          '415256352451336262353342465115544465'  #34
                         )
                     )

In [18]:
Casino_D6 = list(map(int,
                     '4662463662365564236251564246334615654'
                     '2345553452353144416233124244214652563'
                     '6665263655664636341241433116546546323'
                     '3621554253645516622641161563165422222'
                     '3354525311565246532314516351155131433'
                     '5532663231321336362313412143165454552'
                     '5426356555113363546634351162414252234'
                     '5455463414355425616142363325434646546'
                     '2134413334235253166345242544125511566'
                     '4434263226166452122162644235616262343'
                     '2323641555436116511223413555436116635'
                     '3363124125413143516231135553113563631'
                     '1661633331362535636534614224526336546'
                     '3262241135142156133465625464343115653'
                     '1262324315346223312212546242465441666'
                     '2345433536265152436634553462323213161'
                     '6232245321262546633265423326233256532'
                     '2231226122463256666445244556612141623'
                     '5641555666161322646452662556355534146'
                     '4143463645562422616321153545266556523'
                     '6523364421426122341322263256312111642'
                     '3146465652235153633125334113643112244'
                     '3666222213621662415243552352452546643'
                     '5626112362261232243524512426236325121'
                     '5235661351466361642426511322232336133'
                     '6355234155533413153526123161431626445'
                     '4526254542655631454654355456261133152'
                     '1166442632413224321464446256666512316'
                     '3114233352262511623546362165363643655'
                     '532556435521335225156554626224422415'
                     '264656265532143142531151611521263216'
                     '654152634265256566654312221325314253'
                     '516624231665546236155124225233412444'
                     '442213145646236563656532343153152233'
                    )
                )

In [19]:
Large_Chessex_D6 = list(map(int,
                           '2663226632116416142112322152331615546'
                           '6316611215542624611146611452115315124'
                           '4564664142243254341643426145364151364'
                           '2316615525561443321142115631654114566'
                           '4566436151632564615265162163241566521'
                           '2245346426666511356531642623113311636'
                           '2321124645515236123212323321653625413'
                           '2154245422521352556264636136464136642'
                           '3244423216355245126413526615533151661'
                           '6244346654252432333112142435114266165'
                           '5242546646525252523153615514551622551'
                           '2643543344553151632135464214534642133'
                           '4653436665545354366436313162153443135'
                           '5432464314551366246643151121363356521'
                           '6466216426161262454261411532235233163'
                           '2334563351253645335226154564156316211'
                           '1311146643133642536426142116516244661'
                           '3215464213553122455646316544444421543'
                           '1524336323156641511344234564363366251'
                           '1313342654246663415463136623621524122'
                           '5313212265343133146253215152133356443'
                           '1325163132634154361522336452552156135'
                           '2421524122566636211363551625562433633'
                           '2156541254554122563325565365532661214'
                           '4666554321523161344153363532441416642'
                           '4443143241135263113654416311326352252'
                           '1616311362454562524454453334161161231'
                           '2456545565511624442354432116652413531'
                           '4154235231463322444322254442131352413'
                           '141336512143442365564233263612643665'
                           '165522254212312346135511421452651432'
                           '335115166414455446225331262635462516'
                           '631623644563642433443155551511544246'
                           '664135652453324253263231623651233653'
                          )
                      )

In [20]:
Small_Red_Chessex_D6 = list(map(int,
                                '6355533545135255264463254351561555242'
                                '1341123332514456245235656234353134531'
                                '6544352364656411311441323612336111265'
                                '2311462615134312265464116156516132166'
                                '4264524564643431542513622451466434234'
                                '5622544466226461646442516314551466344'
                                '3542216534336254546642613216341213523'
                                '2133462243435551426152312221351655145'
                                '1515513224514634245641351552254146246'
                                '6561431523534163666541434335464353225'
                                '2224411345151112226444544442432161141'
                                '2114425552621665253261453664424612664'
                                '3114463625324424111241561543354313434'
                                '4322323521165616521343244161354444454'
                                '5346355526666556462513264616664116551'
                                '6323113113425615552444353251566146522'
                                '1144362446532231664661664564155166526'
                                '4342212341566254316141114246223555656'
                                '2634343451633124211165631322364536616'
                                '1563441321546352111251124526135116414'
                                '1652623651416536221246321654464622315'
                                '4535126262365652616214516422234423623'
                                '5431443545564156331123453535614623556'
                                '2213434445142465443164223353513661454'
                                '4324645144466551412364625142541215346'
                                '2365564643561442365311261413122541253'
                                '1511212656431444123651442655124154255'
                                '4121333451363534226642662416453211344'
                                '3663322261235422623333555311613356536'
                                '315352336113641246133364453226664532'
                                '455153656132132252644264424433461526'
                                '314455312261131216446256561135566644'
                                '143356125344352255433225136621121365'
                                '566122363162356514456443552362666656'
                               ))

#### Histograms

##### Pink D6

{{ _ = plt.hist(x=rolls, bins=6, color='#0504aa', rwidth=0.85) }}

##### GameScience D6

{{ _ = plt.hist(x=GameScience_D6, bins=6, color='#0504aa', rwidth=0.85) }}

##### Casino D6

{{ _ = plt.hist(x=Casino_D6, bins=6, color='#0504aa', rwidth=0.85) }}

##### Large Chessex D6

{{ _ = plt.hist(x=Large_Chessex_D6, bins=6, color='#0504aa', rwidth=0.85) }}

##### Small Red Chessex D6

{{ _ = plt.hist(x=Small_Red_Chessex_D6, bins=6, color='#0504aa', rwidth=0.85) }}

#### Basic tests

| Dice | $\chi^2$ | Combined $\chi^2$ | Serial 2 | Serial 3 | Gap | Poker | Coupon |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| Pink D6 | {{ p(chi2test(count(rolls, 6))) }} | {{ p(ks_combine_chi2(rolls, 30)) }} | {{ p(serial_test(rolls, 2)) }} | {{ p(serial_test(rolls, 3)) }} | {{ p(gap_test(rolls, 6)) }} | {{ p(poker_test(rolls, 5)) }} | {{ p(coupon_test(rolls, 6)) }} |
| Casino D6 | {{ p(chi2test(count(Casino_D6, 6))) }} | {{ p(ks_combine_chi2(Casino_D6, 30)) }} | {{ p(serial_test(Casino_D6, 2)) }} | {{ p(serial_test(Casino_D6, 3)) }} | {{ p(gap_test(Casino_D6, 6)) }} | {{ p(poker_test(Casino_D6, 5)) }} | {{ p(coupon_test(Casino_D6, 6)) }}|
| GameScience D6 | {{ p(chi2test(count(GameScience_D6, 6))) }} | {{ p(ks_combine_chi2(GameScience_D6, 30)) }} | {{ p(serial_test(GameScience_D6, 2)) }} | {{ p(serial_test(GameScience_D6, 3)) }} | {{ p(gap_test(GameScience_D6, 6)) }} | {{ p(poker_test(GameScience_D6, 5)) }} | {{ p(coupon_test(GameScience_D6, 6)) }}|
| Large Chessex D6 | {{ p(chi2test(count(Large_Chessex_D6, 6))) }} | {{ p(ks_combine_chi2(Large_Chessex_D6, 30)) }} | {{ p(serial_test(Large_Chessex_D6, 2)) }} | {{ p(serial_test(Large_Chessex_D6, 3)) }} | {{ p(gap_test(Large_Chessex_D6, 6)) }} | {{ p(poker_test(Large_Chessex_D6, 5)) }} | {{ p(coupon_test(Large_Chessex_D6, 6)) }}|
| Small Red Chessex D6 | {{ p(chi2test(count(Small_Red_Chessex_D6, 6))) }} | {{ p(ks_combine_chi2(Small_Red_Chessex_D6, 30)) }} | {{ p(serial_test(Small_Red_Chessex_D6, 2)) }} | {{ p(serial_test(Small_Red_Chessex_D6, 3)) }} | {{ p(gap_test(Small_Red_Chessex_D6, 6)) }} | {{ p(poker_test(Small_Red_Chessex_D6, 5)) }} | {{ p(coupon_test(Small_Red_Chessex_D6, 6)) }}|

#### Serial correlation test

| Dice | $2\sigma$ | 1 | 2 | 4 | 6 | 16 | Failures |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| Pink D6  | {{ pc(rolls) }} | {{ cor(rolls, 1) }} | {{ cor(rolls, 2) }} | {{ cor(rolls, 4) }} | {{ cor(rolls, 6) }} | {{ cor(rolls, 16) }} | {{ failures(rolls) }} |
| Casino D6 | {{ pc(Casino_D6) }} | {{ cor(Casino_D6, 1) }} | {{ cor(Casino_D6, 2) }} | {{ cor(Casino_D6, 4) }} | {{ cor(Casino_D6, 6) }} | {{ cor(Casino_D6, 16) }} | {{ failures(Casino_D6) }} |
| GameScience D6 | {{ pc(GameScience_D6) }} | {{ cor(GameScience_D6, 1) }} | {{ cor(GameScience_D6, 2) }} | {{ cor(GameScience_D6, 4) }} | {{ cor(GameScience_D6, 6) }} | {{ cor(GameScience_D6, 16) }} | {{ failures(GameScience_D6) }} |
| Large Chessex D6 | {{ pc(Large_Chessex_D6) }} | {{ cor(Large_Chessex_D6, 1) }} | {{ cor(Large_Chessex_D6, 2) }} | {{ cor(Large_Chessex_D6, 4) }} | {{ cor(Large_Chessex_D6, 6) }} | {{ cor(Large_Chessex_D6, 16) }} | {{ failures(Large_Chessex_D6) }} |
| Small Red Chessex D6 | {{ pc(Small_Red_Chessex_D6) }} | {{ cor(Small_Red_Chessex_D6, 1) }} | {{ cor(Small_Red_Chessex_D6, 2) }} | {{ cor(Small_Red_Chessex_D6, 4) }} | {{ cor(Small_Red_Chessex_D6, 6) }} | {{ cor(Small_Red_Chessex_D6, 16) }} | {{ failures(Small_Red_Chessex_D6) }} |
